# Chatbots with RAG

## 구조 Architectures

1. 챗봇은 일반적으로 도메인 특화 질문에 더 잘 답하기 위해 개인 데이터에 대한 검색 강화 생성, 즉 RAG를 사용
2. 최종 질문 답변을 위해 가장 관련성 높은 맥락만을 사용하도록 여러 데이터 소스 간에 경로를 설정할 수도 있고, 단순히 메시지를 주고받는 것 이상의 더 특화된 유형의 채팅 기록이나 메모리를 사용하기로 선택할 수도 있음.

## 구성 요소

1. Chat Models : 챗봇 인터페이스는 원시 텍스트가 아닌 메시지를 기반으로 하기 때문에 텍스트 LLM보다는 채팅 모델에 더 적합합니다. 여기에서 채팅 모델 통합 목록을 보고, 여기에서 LangChain의 채팅 모델 인터페이스에 대한 문서를 확인하세요. 챗봇에 대해 LLM을 사용할 수도 있지만(여기를 참조), 채팅 모델은 더 대화적인 톤을 가지며 메시지 인터페이스를 기본적으로 지원합니다.

2. Prompt Templates : 기본 메시지, 사용자 입력, 채팅 기록 및 (선택적으로) 추가 검색된 맥락을 결합하는 프롬프트를 조립하는 과정을 단순화함.

3. Chat History : 챗봇이 과거 상호작용을 "기억"하고 후속 질문에 응답할 때 이를 고려할 수 있게 함.

4.  Retrievers(선택사항) : 도메인 특화된 최신 지식을 맥락으로 사용하여 응답을 보강할 수 있는 챗봇을 구축하고 싶을 때 유용함.

In [5]:
import os
os.getcwd()
os.chdir('/workspace')
os.getcwd()

'/workspace'

In [6]:
import dotenv

dotenv.load_dotenv()

# import getpass
# import os

# os.environ["OPENAI_API_KEY"] = getpass.getpass()

True

In [8]:
from langchain_openai import ChatOpenAI

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

In [14]:
from langchain_core.messages import HumanMessage

# 확인 결과 : 불어로 하면 이해 못하고 프랑스어로 해야 이해를 함.

chat.invoke(
    [
        HumanMessage(
            content="이 문장을 프랑스어로 번역해줄래? : 나는 자연어처리를 좋아합니다."
        )
    ]
)

AIMessage(content='Je aime le traitement du langage naturel.', response_metadata={'token_usage': {'completion_tokens': 10, 'prompt_tokens': 41, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-7f079ba9-297c-4516-9172-164bb4692c5c-0')

In [15]:
chat.invoke([HumanMessage(content="뭐라고 말했어?")])

AIMessage(content='죄송합니다, 제가 무엇을 말했는지 기억하지 못합니다. 무엇을 도와드릴까요?', response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 18, 'total_tokens': 62}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-4eb73ae4-ff5d-47c6-9039-b369057f442e-0')

In [17]:
from langchain_core.messages import AIMessage

chat.invoke(
    [
        HumanMessage(
            content="이 문장을 프랑스어로 번역해줄래? : 나는 자연어처리를 좋아합니다."
        ),
        AIMessage(content="Je suis passionné par le traitement du langage naturel."),
        HumanMessage(content="뭐라고 말했어?"),
    ]
)

AIMessage(content='"Je suis passionné par le traitement du langage naturel." 라고 말했습니다. 이것은 "나는 자연어처리를 좋아합니다." 라는 뜻입니다.', response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 73, 'total_tokens': 124}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-42d261f9-36bb-485f-a74c-eb37d5fca104-0')

chat.invoke안에 context를 다 넣어야 전 내용을 받아 이어가는 것으로 보임.

In [19]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", # 'human', 'user', 'ai', 'assistant', or 'system'
            "넌 어시스턴트야. 모든 질문에 답을 해줘야 해.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | chat

In [20]:
chain.invoke(
    {
        "messages": [
            HumanMessage(
                content="이 문장을 프랑스어로 번역해줄래? : 나는 자연어처리를 좋아합니다."
            ),
            AIMessage(content="Je suis passionné par le traitement du langage naturel."),
            HumanMessage(content="What did you just say?"),
        ],
    }
)

AIMessage(content='I said "Je suis passionné par le traitement du langage naturel," which means "I am passionate about natural language processing" in French.', response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 97, 'total_tokens': 127}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_b953e4de39', 'finish_reason': 'stop', 'logprobs': None}, id='run-3aac4a73-d41c-4dac-a4be-7bf8691fd6ed-0')

## Message history

1. 채팅기록 관리를 줄이는 방법론으로, MessageHistory class를 사용

2. 채팅메시지 저장 및 로드 역할.

3. 다양한 데이터베이스에 메시지를 지속시키는 내장 메시지 기록 통합 기능 존재.

4. 하지만, 현재 Jupyter Notebook에서는 메모리 내에서 작동하는 데모 메시지 기록인 ChatMessageHistory를 사용.

In [21]:
from langchain.memory import ChatMessageHistory


demo_chat_history = ChatMessageHistory()
demo_chat_history.add_user_message("hi!")
demo_chat_history.add_ai_message("whats up?")
demo_chat_history.messages

[HumanMessage(content='hi!'), AIMessage(content='whats up?')]

In [23]:
demo_chat_history

InMemoryChatMessageHistory(messages=[HumanMessage(content='hi!'), AIMessage(content='whats up?')])

In [24]:
demo_chat_history.add_user_message(
    "이 문장을 프랑스어로 번역해줄래? : 나는 자연어처리를 좋아합니다."
)

response = chain.invoke({"messages": demo_chat_history.messages})
response

AIMessage(content='당연히요! "나는 자연어처리를 좋아합니다."는 프랑스어로 "J\'aime le traitement du langage naturel."로 번역됩니다.', response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 84, 'total_tokens': 134}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-d04beaf8-3257-424a-b487-5004beae4b85-0')

In [25]:
demo_chat_history.add_ai_message(response)

demo_chat_history.add_user_message("What did you just say?")

chain.invoke({"messages": demo_chat_history.messages})

AIMessage(content='I said, "J\'aime le traitement du langage naturel," which means "I like natural language processing" in French.', response_metadata={'token_usage': {'completion_tokens': 27, 'prompt_tokens': 148, 'total_tokens': 175}, 'model_name': 'gpt-3.5-turbo-1106', 'system_fingerprint': 'fp_482d920018', 'finish_reason': 'stop', 'logprobs': None}, id='run-e2b4c2ff-f84f-4ac2-8833-3fa2358b750a-0')

## Retrievers

1. 챗봇 도메인 특화 지식을 가져오기 위해 Retrievers를 설정할 수 있음.
2. 위의 챗봇을 활용하여 LangSmith에 대한 질문에 답변할 수 있도록 할 수 있음.
3. LangSmith 문서를 소스 자료로 사용하고 나중에 검색할 수 있또록 벡터 저장소에 저장할 것.

In [27]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
data = loader.load()

In [28]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
all_splits = text_splitter.split_documents(data)

In [29]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma.from_documents(documents=all_splits, embedding=OpenAIEmbeddings())

In [30]:
# k is the number of chunks to retrieve
retriever = vectorstore.as_retriever(k=4)

docs = retriever.invoke("how can langsmith help with testing?")

docs

[Document(page_content='Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
 Document(page_conten

Handling documents
Let’s modify our previous prompt to accept documents as context. We’ll use a create_stuff_documents_chain helper function to “stuff” all of the input documents into the prompt, which also conveniently handles formatting. We use the ChatPromptTemplate.from_messages method to format the message input we want to pass to the model, including a MessagesPlaceholder where chat history messages will be directly injected:

In [31]:
from langchain.chains.combine_documents import create_stuff_documents_chain

chat = ChatOpenAI(model="gpt-3.5-turbo-1106")

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user's questions based on the below context:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

In [36]:
from langchain.memory import ChatMessageHistory

demo_chat_history = ChatMessageHistory()

demo_chat_history.add_user_message("LangSmith는 테스트에 어떻게 도움이 될 수 있나요?")

document_chain.invoke(
    {
        "messages": demo_chat_history.messages,
        "context": docs,
    }
)

'LangSmith는 프로덕션급 LLM 애플리케이션을 구축하기 위한 플랫폼으로, 애플리케이션을 밀접하게 모니터링하고 평가하여 신속하고 확신을 갖고 배포할 수 있도록 도와줍니다. LangChain의 사용은 필요하지 않습니다 - LangSmith는 독립적으로 작동합니다. 또한 LangSmith를 사용하여 민감한 데이터를 로깅하지 않고, 팀 내에서만 접근할 수 있도록 보장할 수 있습니다. 자체 호스팅 옵션 또는 기업용 라이선스를 통해 LangSmith를 이용할 수 있습니다. LangSmith는 테스트 및 프로덕션 환경에서 도움을 줄 수 있습니다.'

### Creating a retrieval chain

In [37]:
from typing import Dict

from langchain_core.runnables import RunnablePassthrough


def parse_retriever_input(params: Dict):
    return params["messages"][-1].content


retrieval_chain = RunnablePassthrough.assign(
    context=parse_retriever_input | retriever,
).assign(
    answer=document_chain,
)

In [38]:
response = retrieval_chain.invoke(
    {
        "messages": demo_chat_history.messages,
    }
)

response

{'messages': [HumanMessage(content='LangSmith는 테스트에 어떻게 도움이 될 수 있나요?')],
 'context': [Document(page_content='Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
  Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': '

In [40]:
demo_chat_history.add_ai_message(response["answer"])

demo_chat_history.add_user_message("tell me more about that!")

retrieval_chain.invoke(
    {
        "messages": demo_chat_history.messages,
    },
)

{'messages': [HumanMessage(content='LangSmith는 테스트에 어떻게 도움이 될 수 있나요?'),
  AIMessage(content='LangSmith는 프로덕션급 LLM 응용 프로그램을 구축하기 위한 플랫폼으로, 응용 프로그램을 신속하고 자신 있게 출시할 수 있도록 모니터링 및 평가할 수 있습니다. LangSmith를 사용하면 LangChain이 필요하지 않으며, 독자적으로 작동합니다. 따라서 LangSmith를 사용하여 응용 프로그램의 성능 및 안정성을 테스트하고 개선할 수 있습니다.'),
  HumanMessage(content='tell me more about that!'),
  AIMessage(content='LangSmith는 프로덕션급 LLM 응용 프로그램을 구축하기 위한 플랫폼으로, 응용 프로그램을 신속하고 자신 있게 출시할 수 있도록 모니터링 및 평가할 수 있습니다. LangSmith를 사용하면 LangChain이 필요하지 않으며, 독자적으로 작동합니다. 따라서 LangSmith를 사용하여 응용 프로그램의 성능 및 안정성을 테스트하고 개선할 수 있습니다.'),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content="keySetup your environmentLog your first traceCreate your first evaluationNext StepsAdditional ResourcesFAQHow do I migrate projects between organizations?Why aren't my runs aren't showing up in my project?My team deals with sensitive data that cannot be logged. How can I ensure that only my team can access it?CommunityDiscordTwitterGit

In [41]:
retrieval_chain_with_only_answer = (
    RunnablePassthrough.assign(
        context=parse_retriever_input | retriever,
    )
    | document_chain
)

retrieval_chain_with_only_answer.invoke(
    {
        "messages": demo_chat_history.messages,
    },
)

'LangSmith는 프로덕션급 LLM(언어 모델 라이프사이클) 응용 프로그램을 구축하는 데 도움을 주는 플랫폼입니다. 이를 통해 개발자들은 더 나은 언어 모델을 만들고 평가할 수 있으며, 안정성 및 성능을 향상시킬 수 있습니다. 또한, LangSmith는 자체적으로 작동하여 LangChain에 의존하지 않고 사용할 수 있습니다. 따라서 LangSmith를 사용하여 응용 프로그램의 성능 및 안정성을 테스트하고 개선할 수 있습니다.'

Query transformation
There’s one more optimization we’ll cover here - in the above example, when we asked a followup question, tell me more about that!, you might notice that the retrieved docs don’t directly include information about testing. This is because we’re passing tell me more about that! verbatim as a query to the retriever. The output in the retrieval chain is still okay because the document chain retrieval chain can generate an answer based on the chat history, but we could be retrieving more rich and informative documents:

In [42]:
retriever.invoke("how can langsmith help with testing?")

[Document(page_content='Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
 Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookQuick StartOn this pageGetting started with LangSmithIntroductionâ€‹LangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!Install', metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
 Document(page_conten

In [43]:
retriever.invoke("tell me more about that!")

[Document(page_content="keySetup your environmentLog your first traceCreate your first evaluationNext StepsAdditional ResourcesFAQHow do I migrate projects between organizations?Why aren't my runs aren't showing up in my project?My team deals with sensitive data that cannot be logged. How can I ensure that only my team can access it?CommunityDiscordTwitterGitHubDocs CodeLangSmith SDKPythonJS/TSMoreHomepageBlogLangChain Python DocsLangChain JS/TS DocsCopyright Â© 2024 LangChain, Inc.", metadata={'description': 'Introduction', 'language': 'en', 'source': 'https://docs.smith.langchain.com/overview', 'title': 'Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith'}),
 Document(page_content='lifecycle.Pricing: Learn about the pricing model for LangSmith.Self-Hosting: Learn about self-hosting options for LangSmith.Proxy: Learn about the proxy capabilities of LangSmith.Tracing: Learn about the tracing capabilities of LangSmith.Evaluation: Learn about the evaluation capabilities of Lang

In [44]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableBranch

# We need a prompt that we can pass into an LLM to generate a transformed search query

chat = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.2)

query_transform_prompt = ChatPromptTemplate.from_messages(
    [
        MessagesPlaceholder(variable_name="messages"),
        (
            "user",
            "Given the above conversation, generate a search query to look up in order to get information relevant to the conversation. Only respond with the query, nothing else.",
        ),
    ]
)

query_transforming_retriever_chain = RunnableBranch(
    (
        lambda x: len(x.get("messages", [])) == 1,
        # If only one message, then we just pass that message's content to retriever
        (lambda x: x["messages"][-1].content) | retriever,
    ),
    # If messages, then we pass inputs to LLM chain to transform the query, then pass to retriever
    query_transform_prompt | chat | StrOutputParser() | retriever,
).with_config(run_name="chat_retriever_chain")

In [45]:
document_chain = create_stuff_documents_chain(chat, question_answering_prompt)

conversational_retrieval_chain = RunnablePassthrough.assign(
    context=query_transforming_retriever_chain,
).assign(
    answer=document_chain,
)

demo_ephemeral_chat_history = ChatMessageHistory()

In [46]:
demo_ephemeral_chat_history.add_user_message("how can langsmith help with testing?")

response = conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages},
)

demo_ephemeral_chat_history.add_ai_message(response["answer"])

response

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith can help with testing by providing tracing, evaluation, and production monitoring capabilities for your LLM (Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. With LangSmith, you can trace the behavior of your application, evaluate its performance, and monitor it in production to ensure everything is running smoothly. Additionally, LangSmith offers a Prompt Hub, a prompt management tool built into the platform, which can aid in testing and managing prompts for your language model. If you have specific testing requirements or need further assistance, feel free to reach out to the LangChain team at support@langchain.dev for more information.')],
 'context': [Document(page_content='Getting started with LangSmith | ğŸ¦œï¸�ğŸ›\xa0ï¸� LangSmith', metadata={'description': 'Introduction', 'language': 

In [47]:
demo_ephemeral_chat_history.add_user_message("tell me more about that!")

conversational_retrieval_chain.invoke(
    {"messages": demo_ephemeral_chat_history.messages}
)

{'messages': [HumanMessage(content='how can langsmith help with testing?'),
  AIMessage(content='LangSmith can help with testing by providing tracing, evaluation, and production monitoring capabilities for your LLM (Language Model) applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. With LangSmith, you can trace the behavior of your application, evaluate its performance, and monitor it in production to ensure everything is running smoothly. Additionally, LangSmith offers a Prompt Hub, a prompt management tool built into the platform, which can aid in testing and managing prompts for your language model. If you have specific testing requirements or need further assistance, feel free to reach out to the LangChain team at support@langchain.dev for more information.'),
  HumanMessage(content='tell me more about that!')],
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick Start